In [6]:
import numpy as np
import matplotlib.pyplot as plt

In [39]:
def eroare_relativa(x_nou, x_vechi):
    if x_vechi != 0:
        return np.abs( (x_nou - x_vechi) / x_vechi )
    return None

def metoda_bisectiei(f, a, b, N=None):
    if not N:
        N = int(np.log2((b - a) / epsilon) - 1)
    x_num = (a + b) / 2
    print(f'a0: {a}, b0: {b}, x0: {x_num}')
    for k in range(1, N+1):
        if f(x_num) == 0:
            break
        elif f(a) * f(x_num) < 0:
            b = x_num
        else:
            a = x_num
        aux = x_num
        x_num = (a + b) / 2
        
        print()
        print(f'a{k}: {a}, b{k}: {b}, x{k}: {x_num}')
        print(f'er{k}: {eroare_relativa(x_num, aux)}')
    
    return x_num

def metoda_Newton(f, df, x0, N):
    print(f'x0: {x0}')
    x_num = x0
    for k in range(1, N+1):
        aux = x_num
        x_num = x_num - (f(x_num) / df(x_num))
        
        print()
        print(f'x{k}: {x_num}')
        print(f'er{k}: {eroare_relativa(x_num, aux)}')
        
    return x_num, N

def pozitie_falsa(f, a, b, N):
    x = (a * f(b) - b * f(a)) / (f(b) - f(a))
    print(f'x0: {x}')
    for k in range(1, N+1):
        if abs(f(x)) < 1e-15:
            break
        elif f(a) * f(x) < 0:
            b = x
        elif f(a) * f(x) > 0:
            a = x
        x_urm = (a * f(b) - b * f(a)) / (f(b) - f(a))
        
        print()
        print(f'x{k}: {x_urm}')
        print(f'er{k}: {eroare_relativa(x_urm, x)}')
        
        if abs(x_urm - x) / abs(x) < epsilon:
            break
            
        x = x_urm

    return x, N

def secanta(f, a, b, x0, x1, N):
    print(f'x0: {x0}, x1: {x1}')
    for k in range(1, N+1):
        if x0 == 0:  # verificam sa nu impartim la 0 la pasul urmator
            if abs(f(x0)) < 1e-15:
                return x0, N
            else:
                print('Eroare. Impartire la 0')
                return None, None
        if abs(x1 - x0) / abs(x0) < epsilon:
            break
        x2 = (x0 * f(x1) - x1 * f(x0)) / (f(x1) - f(x0))
        if not a <= x2 <= b:
            print('Introduceti alte valori pentru x0, x1')
            return None, None
        x0 = x1
        x1 = x2
        
        print(f'x0: {x0}, x1: {x1}')
        
    return x1, N

In [18]:
def f(x):
    return x ** 2 + 2 * x - 1
def df(x):
    return 2 * x + 2
epsilon = 1e-5

In [38]:
pozitie_falsa(f, -3, -2, 3)

x0: -2.3333333333333335

x1: -2.4
er1: 0.028571428571428466

x2: -2.411764705882353
er2: 0.004901960784313708

x3: -2.413793103448276
er3: 0.0008410428931875814


(-2.413793103448276, 3)

In [32]:
def gauss(A):
    if np.linalg.det(A) < 1e-15:
        print('determinantul e 0')
        return np.array([])
    n = A.shape[0]
    for k in range(n-1):
        for l in range(k+1, n):
            A[l] = A[l] - (A[l,k] / A[k,k]) * A[k]
    return A

def gauss_pp(A, eps=1e-15):
    delta = np.linalg.det(A)
    if abs(delta) < eps:
        print("Sistemul nu are solutie unica")
        return None, None
    n = A.shape[0]
    P = np.identity(n)
    U = A.copy()
    for k in range(n-1):
        p = np.argmax(abs(A[k:,k])) + k
        P[p], P[k] = P[k].copy(), P[p].copy()
        U[p], U[k] = U[k].copy(), U[p].copy()
        U[k+1:,k:] = U[k+1:,k:] - np.outer(U[k+1:,k] / U[k,k], [U[k,k:]])
    return P, U

def lu(A):
    n = A.shape[0]
    U = A.copy()
    L = np.identity(n)
    for k in range(n - 1):
        L[k+1:,k] = U[k+1:, k] / U[k, k]
        U[k+1:, k:] -= np.outer(L[k+1:, k], U[k,k:])
        print()
        print(L)
        print(U)
    return L, U

In [40]:
def gauss_pt(A, b, eps=1e-13):
    delta = np.linalg.det(A)
    if abs(delta) < eps:
        print("Sistemul nu are solutie unica")
        return None, None, None
    
    n = A.shape[0]
    P = np.identity(n)
    U = A.copy()
    c = b.copy()
    ordine = np.array(range(n))
    for k in range(n-1):
        # aflam maximul din matricea din dreapta jos. functia ne returneaza numarul elementului 
        p = np.argmax(abs(U[k:,k:]))
        pi = p // (n-k) + k  # calculam linia si coloana. (n-k) e nr de elemente de pe o linie din matricea selectata
        pj = p % (n-k) + k
        
        # schimbam liniile intre ele
        P[pi], P[k] = P[k].copy(), P[pi].copy()
        U[pi], U[k] = U[k].copy(), U[pi].copy()
        c[pi], c[k] = c[k].copy(), c[pi].copy()
        
        # schimbam coloanele pentru U si memoram schimbarea in `ordine`
        U[:,pj], U[:,k] = U[:,k].copy(), U[:,pj].copy()
        ordine[pj], ordine[k] = ordine[k], ordine[pj]
        
        for l in range(k+1, n):
            c[l] = c[l] - (U[l,k] / U[k,k]) * c[k]
            U[l] = U[l] - (U[l,k] / U[k,k]) * U[k]
            
    return P, U, c, ordine

def get_x_gauss_pt(U, b, ordine):
    n = U.shape[0]
    y = np.zeros((n,1))
    x = np.zeros((n,1))
    for i in range(n - 1, -1, -1):
        x[i,0] = (b[i] - U[i,i+1:]@x[i+1:]) / U[i,i]
        
    # aplicam permutarea pe x pentru ca solutiile sunt in alta ordine
    x_ = np.zeros((n,1))
    for i in range(n):
        x_[ordine[i],0] = x[i,0]
    
    return x_

In [43]:
A = np.array([[2., -1., -2.],
              [4., 2., 0.],
              [0., -2., -1.]])
b = np.array([[1.],
              [1.],
              [1.]])

In [44]:
P, U, c, ordine = gauss_pt(A, b)
if U is not None:
    x = get_x_gauss_pt(U, c, ordine)
    print(x)

[[ 0.625]
 [-0.75 ]
 [ 0.5  ]]
